In [ ]:
import pyvista as pv

pv.set_jupyter_backend("static")

%load_ext autoreload
%autoreload 2

# Constitutive Models: Deep Dive

In [ ]:
import logging
logging.basicConfig()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from materialite import (
    Material,
    Box,
    Scalar,
    Order2SymmetricTensor,
    Order4SymmetricTensor,
    Orientation,
    SlipSystem,
)
from materialite.models.small_strain_fft import (
    SmallStrainFFT,
    LoadSchedule,
    Elastic,
    ElasticViscoplastic,
    voce,
)

Here, we will take a deeper dive into some of the capabilities of `SmallStrainFFT`, including output quantities, postprocessing, and simulation details. Start with the `Material` from Example 3 in the constitutive models demo.

In [ ]:
num_grains = 30
grains_list = np.arange(num_grains)
rng = np.random.default_rng(1)

material = (
    Material(dimensions=[16, 16, 16])
    .create_voronoi(num_grains, periodic=True, label="grain", rng=rng)
    .assign_random_orientations(
        region_label="grain", orientation_label="orientation", rng=rng
    )
)

**Example 1: Visualizing different output quantities**

Run a uniaxial tension simulation out to 1% strain.

In [ ]:
stiffness_tensor = Order4SymmetricTensor.from_cubic_constants(
    C11=243300, C12=156700, C44=117800
)
constitutive_model = ElasticViscoplastic(
    stiffness=stiffness_tensor,
    slip_systems=SlipSystem.octahedral(),
    reference_slip_rate=1.0,
    rate_exponent=10.0,
    slip_resistance=143.0,
    hardening_function=voce,
    hardening_properties={"tau_1": 50.0, "theta_0": 1450.0, "theta_1": 95.0},
)
load_schedule = LoadSchedule.from_constant_uniaxial_strain_rate(
    magnitude=1.0, direction="z"
)
model = SmallStrainFFT(
    load_schedule=load_schedule,
    end_time=0.01,
    initial_time_increment=0.001,
    constitutive_model=constitutive_model,
)

output_times = np.linspace(start=0.001, stop=0.01, num=10)
material_1 = material.run(
    model,
    output_times=output_times,
    output_variables=[
        "slip_system_shear_strains",
        "plastic_strains",
        "slip_resistances",
    ],
)

The material will now have fields called "stress", "strain", "plastic_strains", "slip_system_shear_strains", and "slip_resistances". Let's plot some of these quantities.

Mean stress-strain curve in the loading direction:

In [ ]:
stress = material_1.extract("stress")
strain = material_1.extract("strain")
fig, ax = plt.subplots()
ax.plot(strain.mean("p").components[:, 2], stress.mean("p").components[:, 2])
ax.set_xlabel("Axial strain [-]")
ax.set_ylabel("Axial stress [MPa]")

Mean plastic strain in the loading direction versus total strain:

In [ ]:
plastic_strain = material_1.extract("plastic_strains")
strain = material_1.extract("strain")
mean_strain = strain.mean("p").components[:, 2]
fig, ax = plt.subplots()
ax.plot(mean_strain, plastic_strain.mean("p").components[:, 2])
ax.set_xlabel("Axial strain [-]")
ax.set_ylabel("Axial plastic strain [-]")

Pointwise equivalent plastic strain at the end of the simulation:

In [ ]:
eq_plastic_strain = plastic_strain.dev.norm * np.sqrt(2 / 3)
material.create_fields({"eq_plastic_strain": eq_plastic_strain}).plot(
    "eq_plastic_strain"
)

Pointwise CRSS. Since the hardening law is a function of only the accumulated slip, the CRSS is the same on all slip systems at a given point. Here, we take the max over the slip systems, but taking the mean would yield the same result in this case.

In [ ]:
material_1.create_fields(
    {"max_slip_resistance": material_1.extract("slip_resistances").max("s")}
).plot("max_slip_resistance")

We can do the same with the max slip across the 12 slip systems at each point.

In [ ]:
material_1.create_fields(
    {"max_slip": material_1.extract("slip_system_shear_strains").abs.max("s")}
).plot("max_slip", color_lims=(0.002, 0.02))

We can also calculate and plot grain-average quantities. The method `get_region_indices` returns a dictionary whose keys are the region IDs (in this case grain IDs) and whose values are the indices in `material.get_fields()` corresponding to each region ID.

In [ ]:
grain_indices_lookup = material.get_region_indices("grain")
grain_avg_eq_stresses = Scalar.from_list(
    [
        stress[indices].mean().dev.norm * np.sqrt(1.5)
        for indices in grain_indices_lookup.values()
    ]
)
regional_field = {
    "grain": grain_indices_lookup.keys(),
    "grain_avg_eq_stress": grain_avg_eq_stresses,
}
material_1.create_regional_fields(
    region_label="grain", regional_fields=regional_field
).plot("grain_avg_eq_stress")

**Example 2: Cyclic loading**

Here, we apply one strain-controlled cycle with a frequency of 1 Hz. We request stress and strain output every 1/8 of a cycle. Other desired state variables could be passed to `material.run(...)` as in Example 1. Requesting the `INFO` outputs allows us to track the automatic time incrementation. The time increment is doubled when the material model converges in 10 or fewer iterations. The time increment is reduced when the material model fails to converge, or the time increment must be shortened to match a requested output time.

In [ ]:
strain_amplitude = Order2SymmetricTensor.from_strain_voigt([0.01, 0, 0, 0, 0, 0])
stress_mask = np.array([0, 1, 1, 1, 1, 1])
load_schedule = LoadSchedule.from_cyclic_strain(
    strain_amplitude=strain_amplitude, frequency=1.0, stress_mask=stress_mask
)

model = SmallStrainFFT(
    load_schedule=load_schedule,
    end_time=1.0,
    initial_time_increment=0.01,
    max_time_increment=0.125,
    constitutive_model=constitutive_model,
)
num_output_times = 8
output_times = np.linspace(start=0.125, stop=1.0, num=num_output_times)

logger = logging.getLogger()
logger.setLevel(logging.INFO)
material_2 = material.run(model, output_times=output_times)
logger.setLevel(logging.WARNING)

Plot the axial strain over time to verify the cyclic loading was applied correctly.

In [ ]:
strain = material_2.extract("strain")
mean_strain = strain.mean("p").components[:, 0]
fig, ax = plt.subplots()
ax.plot(output_times, mean_strain)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Axial strain [-]")

Plot the axial stress-strain curve. Note that the stress relaxes slightly due to the viscoplastic material model and low strain rate at the maximum and minimum of the cycle. The stress relaxation can be reduced by increasing the rate exponent to more closely approximate rate-independent behavior.

In [ ]:
stress = material_2.extract("stress")
mean_stress = stress.mean("p").components[:, 0]
fig, ax = plt.subplots()
ax.plot(mean_strain, mean_stress)
ax.set_xlabel("Axial strain [-]")
ax.set_ylabel("Axial stress [MPa]")

**Example 3: Breaking periodic boundary conditions**

Spectral solution methods like the one in `SmallStrainFFT` inherently have periodic boundary conditions. Adding a zero-stiffness phase to the surfaces of the domain allows for stress-free boundary conditions to instead be applied to the surfaces of the solid material. We accomplish this using a `Box` feature and adding it to the `Material`. The `Box` feature assigns a phase ID of 0 to the first 5 layers of points in the x and y directions, starting from the origin.

Note that, if you want to do anything with grain-average quantities later, a new grain ID value (different from all of the "real" grains) should also be assigned to the `Box`es. Here, we choose -1. We also need to add a corresponding `Orientation` to the regional field that assigned `Orientation`s to each grain. The `Orientation` is arbitrary; we use the identity here for convenience.

In [ ]:
zero_stiffness_phase = Box(
    min_corner=[material.origin, material.origin],
    max_corner=[[4, np.inf, np.inf], [np.inf, 4, np.inf]],
)
grain_field_update = {"grain": [-1], "orientation": [Orientation.identity()]}
material_3 = (
    material.create_uniform_field("phase", 1)
    .update_regional_field("grain", grain_field_update)
    .insert_feature(zero_stiffness_phase, fields={"phase": 0, "grain": -1})
)
material_3.plot("phase", show_edges=True)

Running a model follows the same steps as Example 4 in the constitutive models demo. Create a regional field to assign constitutive models to the zero-stiffness phase (phase ID 0) and the solid material (phase ID 1).

In [ ]:
pore_model = Elastic(Order4SymmetricTensor.zero())
phase_field = {"phase": [0, 1], "constitutive_model": [pore_model, constitutive_model]}
material_3 = material_3.create_regional_fields(region_label="phase", regional_fields=phase_field)
load_schedule = LoadSchedule.from_constant_uniaxial_strain_rate(
    magnitude=1.0, direction="z"
)
model = SmallStrainFFT(
    load_schedule=load_schedule,
    end_time=0.01,
    initial_time_increment=0.001,
)
output_times = np.linspace(start=0.001, stop=0.01, num=10)
material_3 = material_3.run(
    model, output_times=output_times, phase_label="phase"
)

Plot the stress in the loading direction at the end of the simulation.

In [ ]:
material_3.plot("stress", component=2)

If you want to simulate microstructure in the center of a specimen, you could use an `IsotropicElasticPlastic` buffer material instead of `Elastic` with zero stiffness.

**Example 4: Using a postprocessor on state variables stored during a simulation**

For a simulation with a large `Material` and outputs at many time points, the memory footprint can be large. To mitigate this issue, a `postprocessor` function can be passed to the `run` method of `SmallStrainFFT`.

At each time step where output is requested, `SmallStrainFFT` will pass a dictionary to `postprocessor`. The keys of the dictionary are "stress", "strain", and any additional state variable outputs requested by the user (e.g., "plastic_strains", "slip_resistances", and/or "slip_system_shear_strains" for an `ElasticViscoplastic` constitutive model). The values of the dictionary will be pointwise Materialite `Tensor`s. `postprocessor` must output a dictionary, but the keys and values are arbitrary.

Here, the postprocessor will simply compute the mean stress and strain in the loading direction at each time step.

In [ ]:
def postprocessor(outputs):
    mean_strain = outputs["strain"].mean("p").components[2]
    mean_stress = outputs["stress"].mean("p").components[2]
    return {"mean_stress": mean_stress, "mean_strain": mean_strain}

Run the same model as in Example 1.

In [ ]:
stiffness_tensor = Order4SymmetricTensor.from_cubic_constants(
    C11=243300, C12=156700, C44=117800
)
constitutive_model = ElasticViscoplastic(
    stiffness=stiffness_tensor,
    slip_systems=SlipSystem.octahedral(),
    reference_slip_rate=1.0,
    rate_exponent=10.0,
    slip_resistance=143.0,
    hardening_function=voce,
    hardening_properties={"tau_1": 50.0, "theta_0": 1450.0, "theta_1": 95.0},
)
load_schedule = LoadSchedule.from_constant_uniaxial_strain_rate(
    magnitude=1.0, direction="z"
)
model = SmallStrainFFT(
    load_schedule=load_schedule,
    end_time=0.01,
    initial_time_increment=0.001,
    constitutive_model=constitutive_model,
)

output_times = np.linspace(start=0.001, stop=0.01, num=10)
material_4 = material.run(
    model, output_times=output_times, postprocessor=postprocessor
)

Now, the outputs are stored in `material_4.state`, which is a dictionary with the same keys as the one returned by `postprocessor` (here, "mean_stress" and "mean_strain"). The values of the dictionary are lists of the values computed by `postprocessor` at each output time.

In [ ]:
material_4.state

Plot the mean stress-strain curve. This is identical to the one from Example 1.

In [ ]:
fig, ax = plt.subplots()
ax.plot(material_4.state["mean_strain"], material_4.state["mean_stress"])
ax.set_xlabel("Axial strain [-]")
ax.set_ylabel("Axial stress [-]")

`SmallStrainFFT` still adds the stress and strain fields at the last time step as fields.

In [ ]:
material_4.get_fields()

As an example of a more complex operation, suppose we want the grain-average stresses at each time increment. The postprocessing function will use the indices corresponding to each grain (see the last part of Example 1) to get the grain-average stresses from the outputs dictionary.

First, set up a function that takes both the outputs and lists of indices corresponding to each grain as inputs.

In [ ]:
def postprocessor_base(outputs, grain_indices):
    stress = outputs["stress"]
    grain_avg_eq_stresses = [
        stress[indices].mean().dev.norm * np.sqrt(1.5) for indices in grain_indices
    ]
    return {
        "grain_avg_eq_stress": Scalar.from_stack(grain_avg_eq_stresses, new_dim="g")
    }

Next, use Python's built-in `partial` function to "freeze" the `grain_indices` argument in the function we created.

In [ ]:
from functools import partial

grain_indices_lookup = material.get_region_indices("grain")
postprocessor = partial(postprocessor_base, grain_indices=grain_indices_lookup.values())
output_times = np.linspace(start=0.001, stop=0.01, num=10)
material_4b = material.run(
    model, output_times=output_times, postprocessor=postprocessor
)

The state dictionary will now contain a single key, "grain_avg_eq_stress", whose value is a list of the 30 grain-average stresses at each of the 10 requested output times.

In [ ]:
print(material_4b.state.keys())
print(len(material_4b.state["grain_avg_eq_stress"]))

Look at the grain-average stresses at the end of the simulation.

In [ ]:
material_4b.state["grain_avg_eq_stress"][-1]

If we add the grain-average stresses at the end of the last time increment to the `Material` as a regional field, we will get the same plot as in Example 1.

In [ ]:
grain_avg_eq_stresses = Scalar.from_stack(
    material_4b.state["grain_avg_eq_stress"], new_dim="t", axis=1
)
regional_field = {
    "grain": grain_indices_lookup.keys(),
    "grain_avg_eq_stress": grain_avg_eq_stresses,
}
material_4b = material_4b.create_regional_fields("grain", regional_field)
material_4b.plot("grain_avg_eq_stress")

**Other notes**

There are three convergence tolerances in `SmallStrainFFT`. The two tolerances for the global Newton-Raphson solver in nonlinear problems are based on the Abaqus/Implicit solver, adapted for the solution variables in the FFT-based solver:

* `global_tolerance`: tolerance on the maximum equilibrium error at the end of an iteration. The default is $0.001$. The equilibrium error at a point is $R = \mathcal{F}^{-1} \left[\mathbb{G} \mathcal{F}\left( \mathbf{\sigma} \right) \right] / \bar{\sigma}_\text{vm}$, where $\mathcal{F}$ is the Fourier transform, $\mathbf{\sigma}$ is the Cauchy stress tensor at the point, $\bar{\sigma}_\text{vm}$ is the mean von Mises stress in the domain, and $\mathbb{G}$ is a projection tensor that enforces equilibrium (see [1] for details). The maximum equilibrium error is then $r = \max (R)$, where the maximum is over all components (xx, yy, zz, yz, xz, xy) at all points.

* `strain_correction_tolerance`: tolerance on the maximum normalized strain correction at the end of an iteration. The default is $0.01$. The normalized strain correction at a point is $C = \mathbb{D} \mathbf{\varepsilon^*} / \max \left(\Delta \sigma_\text{vm} \right)$, where $\mathbb{D}$ os the consistent tangent at the point, $\mathbf{\varepsilon^*}$ is the strain correction in the current iteration, and $\max \left(\Delta \sigma_\text{vm} \right)$ is the maximum von Mises stress increment in the domain for the current time step. The strain correction is mapped into stress space using $\mathbb{D}$ in order to avoid unnecessary iterations when there is a low- or zero-stiffness phase in the domain. The maximum normalized correction is then $c = \max (C)$, where the maximum is over all components at all points.

At the end of each global Newton-Raphson iteration, the solution is accepted if both the equilibrium and strain correction tolerances are satisfied. Again following Abaqus, the solution is also accepted if the equilibrium tolerance is satisfied, and the *projected* strain correction for the next iteration satisfies the tolerance. The projected strain correction is estimated as $c_{i+1} \approx c_i r_i / r_{i-1}$, where $c_i$ is the maximum normalized strain correction in the current iteration, and $r_i$ and $r_{i-1} are the maximum equilibrium error in the current and previous iteration, respectively.

Finally, determining the strain correction in each iteration requires solving a linear system of the form $\mathcal{A}\left(\mathbf{\varepsilon^*}\right) = \mathbf{b}$. The original formulation in [1] used conjugate gradients to solve this equation; `SmallStrainFFT` uses the MINRES solver in scipy, which allows a solution to be found in cases with an infinite stiffness mismatch (see [2] for details). `linear_solver_tolerance` is the tolerance for the MINRES iterative solver. The default is $1 \times 10^{-5}$.

Any of these tolerances can be set by the user by passing keyword arguments to when the simulation is run (e.g., `material.run(model, global_tolerance=...)` or `model(material, global_tolerance=...)`).

IMPORTANT: for linear problems (i.e., `SmallStrainFFT` with `Elastic` constitutive model(s) at all points), we suggest setting the `linear_solver_tolerance` to a small value, at most the default, and the Newton-Raphson tolerances to arbitrarily large values. This is because the Newton-Raphson iterations are meaningless in the linear case, and the error is entirely controlled by the MINRES solver.

[1] J. Zeman, T.W.J. de Geus, J. Vondrejc, R.H.J. Peerlings, M.G.D. Geers. A finite element perspective on nonlinear FFT-based micromechanical simulations. International Journal for Numerical Methods in Engineering, 2017, 111(10):903–926. doi: 10.1002/nme.5481

[2] S. Lucarini, L. Cobian, A. Voitus, J. Segurado. Adaptation and validation of FFT methods for homogenization of lattice based materials. Computer Methods in Applied Mechanics and Engineering, 2022, 388:114223. doi: 10.1016/j.cma.2021.114223